### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

In [4]:
print(nyt_api_key)
print(tmdb_api_key)

1b5zds6gWGokXaLgBHuj9hd6jSCI66lE
a3e37d80b0dd625b03450572d8c48416


### Access the New York Times API

In [5]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
api_key = "1b5zds6gWGokXaLgBHuj9hd6jSCI66lE" 

query_url = (f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&sort={sort}&fl={field_list}")

print(query_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=1b5zds6gWGokXaLgBHuj9hd6jSCI66lE&begin_date=20130101&end_date=20230531&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count


In [6]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(3):
    # create query with a page number
    page_url = f"{query_url}&page={page}"
    # API results show 10 articles at a time
    print(f"Retrieving page {page} of reviews...")
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_url)
    reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)

        # Print the page that was just retrieved
        print(f"Page {page} retrieved") 

    except KeyError:
        # Print the page number that had no results then break from the loop
        print(f"No results on page {page}")
        break

Retrieving page 0 of reviews...


Page 0 retrieved
Retrieving page 1 of reviews...
Page 1 retrieved
Retrieving page 2 of reviews...
Page 2 retrieved


In [7]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [8]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df = pd.json_normalize(reviews_list)

In [9]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
df['title'] = df['headline.main'].str.extract("\u2018(.+?)\u2019 Review")

In [10]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df['keywords'] = df['keywords'].apply(extract_keywords)

In [11]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = df['title'].to_list()

### Access The Movie Database API

In [12]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 0


# Loop through the titles


    # Check if we need to sleep before making a request

    # Add 1 to the request counter
    
    
    # Perform a "GET" request for The Movie Database
    

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    
        # Get movie id
       

        # Make a request for a the full movie details

        # Execute "GET" request with url
        
        
        # Extract the genre names into a list


        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list



        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        

        
        # Print out the title that was found
        
    

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data


In [ ]:
# Convert the results to a DataFrame


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


In [ ]:
# Export data to CSV without the index
